In [93]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import f1_score
from sklearn.metrics import classification_report

In [94]:
data_train = pd.read_csv('../ML/datasets/adult_data_train.csv')
data = pd.read_csv('../ML/datasets/adult_data_reserved.csv')

In [95]:
data_train.replace('?', pd.NA, inplace=True)
data.replace('?', pd.NA, inplace=True)

In [96]:
data_train.dropna(inplace=True)
data_train.reset_index(drop=True, inplace=True)
data_train.fillna(data.mode().iloc[0], inplace=True)

In [97]:
data.dropna(inplace=True)
data.reset_index(drop=True, inplace=True)

In [98]:
for column in data.columns:
    print(f'Processing column: {column}')
    for i, value in enumerate(data[column]):
      if value not in data_train[column].values:
        most_frequent_value = data[column].mode()[0]
        data.at[i, column] = most_frequent_value
data.columns

Processing column: age
Processing column: workclass
Processing column: fnlwgt
Processing column: education
Processing column: education-num
Processing column: marital-status
Processing column: occupation
Processing column: relationship
Processing column: race
Processing column: sex
Processing column: capital-gain
Processing column: capital-loss
Processing column: hours-per-week
Processing column: native-country


Index(['age', 'workclass', 'fnlwgt', 'education', 'education-num',
       'marital-status', 'occupation', 'relationship', 'race', 'sex',
       'capital-gain', 'capital-loss', 'hours-per-week', 'native-country'],
      dtype='object')

In [100]:
le = LabelEncoder()
str_columns = ['workclass', 'education', 'marital-status', 'occupation', 'relationship', 'race', 'sex', 'native-country']
for i in str_columns:
	le.fit(data_train[i])
	data_train[i] = le.transform(data_train[i])
for i in str_columns:
	le.fit(data[i])
	data[i] = le.transform(data[i])

In [101]:
X = data_train.drop(columns = 'label')
y = data_train['label']

In [104]:
scaler = MinMaxScaler()
scaler.fit(X)
X_train_scaler = scaler.transform(X)
X_test_scaler = scaler.transform(data)

In [105]:
knn = KNeighborsClassifier()
knn.fit(X_train_scaler, y)
y_pred = knn.predict(X_test_scaler)
y_pred

array([0, 1, 1, ..., 1, 0, 0], dtype=int64)

In [108]:
with open('../ML/outputs/file_adultReserve.txt', 'w') as f:
    f.write(f'{list(y_pred)}')